# Proyecto Extracción Estructurada de Texto

## Bloque 0: Instalación de Librerías

En este bloque se instalan todas las dependencias necesarias para el proyecto de estructuración de texto OCR de facturas usando un modelo DistilBERT con un enfoque RAG. Se incluyen librerías para procesamiento de texto, modelado con transformers, manejo de datos, evaluación, visualización y recuperación de contexto.

In [ ]:
# Instalación de librerías clave para NLP, RAG y evaluación
# Ejecutar en entorno basado en Python 3.13+

# Transformers y tokenización
!pip install --upgrade transformers

# Torch (compatible con CUDA si está disponible)
!pip install --upgrade torch torchvision torchaudio

# FAISS para recuperación (versión CPU por simplicidad)
!pip install faiss-cpu

# Pandas y visualización
!pip install pandas matplotlib seaborn

# Scikit-learn para métricas y split de datos
!pip install scikit-learn

# Datasets de Hugging Face (opcional pero útil)
!pip install datasets

# tqdm para barras de progreso
!pip install tqdm

# json5 para análisis robusto de JSON
!pip install json5

# rapidfuzz para comparación de texto (opcional)
!pip install rapidfuzz

!pip install rank_bm25


   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
    --------------------------------------- 2.6/109.3 MB 15.5 MB/s eta 0:00:07
   -- ------------------------------------- 6.6/109.3 MB 17.0 MB/s eta 0:00:07
   --- ------------------------------------ 10.7/109.3 MB 18.5 MB/s eta 0:00:06
   ----- ---------------------------------- 14.7/109.3 MB 18.6 MB/s eta 0:00:06
   ------ --------------------------------- 18.4/109.3 MB 18.4 MB/s eta 0:00:05
   -------- ------------------------------- 23.1/109.3 MB 19.1 MB/s eta 0:00:05
   --------- ------------------------------ 26.7/109.3 MB 19.5 MB/s eta 0:00:05
   ----------- ---------------------------- 31.2/109.3 MB 19.5 MB/s eta 0:00:05
   ------------- -------------------------- 36.7/109.3 MB 20.1 MB/s eta 0:00:04
   --------------- ------------------------ 41.2/109.3 MB 20.5 MB/s eta 0:00:04
   ----------------- ---------------------- 46.7/109.3 MB 21.0 MB/s eta 0:00:03
   ------------------ --------------------- 51.6/10

## Bloque 1: Importación y Setup Inicial

Este bloque importa todas las librerías necesarias para el proyecto y realiza una verificación del entorno, incluyendo versión de Python, disponibilidad de GPU con PyTorch y confirmación de rutas clave. Este setup es esencial para asegurar que las siguientes etapas puedan ejecutarse correctamente.

In [4]:
import os
import sys
import json
import json5
import torch
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

from transformers import DistilBertTokenizerFast, DistilBertModel, DistilBertForSequenceClassification
from transformers import EncoderDecoderModel, Trainer, TrainingArguments

import faiss                    # Para recuperación eficiente (RAG)
from rapidfuzz import fuzz      # Para comparación opcional de texto

# Configuración visual
sns.set(style="whitegrid")

# Verificación del entorno
print(f"Python versión: {sys.version}")
print(f"PyTorch versión: {torch.__version__}")
print(f"GPU disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Usando dispositivo: {torch.cuda.get_device_name(0)}")
else:
    print("GPU no disponible. Se usará CPU.")


Python versión: 3.13.2 (tags/v3.13.2:4f8bb39, Feb  4 2025, 15:23:48) [MSC v.1942 64 bit (AMD64)]
PyTorch versión: 2.9.0+cpu
GPU disponible: False
GPU no disponible. Se usará CPU.


## Bloque 2: Carga del Dataset CSV

En este bloque se carga el archivo CSV que contiene el texto OCR y los datos estructurados en formato JSON. Se muestran algunos ejemplos para asegurar que el formato de entrada sea correcto, y se filtran únicamente las columnas necesarias para el entrenamiento del modelo: OCRed Text y Json Data.

In [8]:
# Cargar el dataset de facturas OCR con estructura JSON
import pandas as pd

# Ruta al archivo cargado (ajustada para entorno de notebook)
csv_path = "./Dataset/facturas.csv"

# Leer el archivo completo
df_raw = pd.read_csv(csv_path)

In [10]:
df_raw.head()

,File Name,Json Data,OCRed Text
0,batch1-0494.jpg,"\n{\n ""invoice"": {\n ""client_name"": ""Clark...",Invoice no: 84652373 Date of issue: 02/23/2021...
1,batch1-0489.jpg,"\n{\n ""invoice"": {\n ""client_name"": ""Willi...",Invoice no: 37451664 Date of issue: 06/11/2020...
2,batch1-0499.jpg,"\n{\n ""invoice"": {\n ""client_name"": ""Heste...",Invoice no: 40108666 Date of issue: 02/07/2020...
3,batch1-0497.jpg,"\n{\n ""invoice"": {\n ""client_name"": ""Olson...",Invoice no: 73285932 Date of issue: 07/25/2017...
4,batch1-0081.jpg,"\n{\n ""invoice"": {\n ""client_name"": ""Wilso...",Invoice no: 15288019 Date of issue: 09/07/2014...


In [11]:
# Verificar las columnas disponibles
print("Columnas del archivo CSV:")
print(df_raw.columns.tolist())

# Filtrar columnas necesarias
df = df_raw[['OCRed Text', 'Json Data']].copy()
df.columns = ['ocr_text', 'json_target']  # Renombrar para facilidad

# Mostrar tamaño del dataset
print(f"\nRegistros cargados: {len(df)}")

# Mostrar ejemplos
print("\nEjemplo de texto OCR:")
print(df['ocr_text'].iloc[0][:500])  # Mostrar primeros 500 caracteres

print("\nJSON estructurado correspondiente:")
print(df['json_target'].iloc[0])


Columnas del archivo CSV:
['File Name', 'Json Data', 'OCRed Text']

Registros cargados: 1414

Ejemplo de texto OCR:
Invoice no: 84652373 Date of issue: 02/23/2021 Seller: Client: Nguyen-Roach Clark-Foster 247 David Highway 77477 Cliff Apt. 853 Lake John, WV 84178 Washingtonbury, MS 78346 Tax Id: 991-72-5826 Tax Id: 937-70-8530 IBAN: GB91/YXO05542456978150 ITEMS No. Description Qty UM Net price Net worth VAT [%] Gross worth Stemware Rack Display Kitchen 1,00 each 42,32 42,32 10% 46,55 Wine Glass Holder Bottle Carbon Steel Free Punch 2 VTG (4) 7 Ounce Since 1852 1,00 each 14,00 14,00 10% 15,40 Milk Bottle Wine 

JSON estructurado correspondiente:

{
  "invoice": {
    "client_name": "Clark-Foster",
    "client_address": "77477 Troy Cliff Apt. 853\nWashingtonbury, MS 78346",
    "seller_name": "Nguyen-Roach",
    "seller_address": "247 David Highway\nLake John, WV 84178",
    "invoice_number": "84652373",
    "invoice_date": "02/23/2021",
    "due_date": ""
  },
  "items": [
    {
      "

## Bloque 3: División del Dataset

En este bloque se divide el dataset en tres subconjuntos: entrenamiento (80%), validación (10%) y prueba (10%). Esta división permite entrenar el modelo, ajustar hiperparámetros y finalmente evaluar su capacidad de generalización. También se muestran ejemplos y estadísticas de cada subconjunto.

In [13]:
# División del dataset en train, validation y test

from sklearn.model_selection import train_test_split

# Dividir en train (70%) y temp (30%)
df_train, df_temp = train_test_split(
    df,
    test_size=0.20,
    random_state=42,
    shuffle=True
)

# Dividir temp en validation (15%) y test (15%) → proporción sobre total
df_val, df_test = train_test_split(
    df_temp,
    test_size=0.50,
    random_state=42,
    shuffle=True
)

# Mostrar tamaño de cada subconjunto
print(f"Tamaño del conjunto de entrenamiento: {len(df_train)}")
print(f"Tamaño del conjunto de validación: {len(df_val)}")
print(f"Tamaño del conjunto de prueba: {len(df_test)}")

# # Mostrar ejemplos de cada subconjunto
# print("\nEjemplo del conjunto de entrenamiento:")
# print(df_train.iloc[0]['ocr_text'][:300])  # Primeros 300 caracteres
# print("\nJSON estructurado:")
# print(df_train.iloc[0]['json_target'])

# print("\nEjemplo del conjunto de validación:")
# print(df_val.iloc[0]['ocr_text'][:300])
# print("\nJSON estructurado:")
# print(df_val.iloc[0]['json_target'])

# print("\nEjemplo del conjunto de prueba:")
# print(df_test.iloc[0]['ocr_text'][:300])
# print("\nJSON estructurado:")
# print(df_test.iloc[0]['json_target'])


Tamaño del conjunto de entrenamiento: 1131
Tamaño del conjunto de validación: 141
Tamaño del conjunto de prueba: 142


## Bloque 4: Creación del RAG Retriever

En este bloque se construye un motor de recuperación basado en BM25 utilizando las instancias del conjunto de entrenamiento (json_target). Cada JSON estructurado se transforma en un documento indexado. Se implementa la función retrieve_similar_example(text) que, dado un nuevo texto OCR, devuelve el JSON más relevante como contexto de soporte para el modelo.

In [24]:
# BM25 Retriever basado en json_target de entrenamiento
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk

# Descargar tokenizer si no se ha hecho antes
nltk.download('punkt')

# Tokenizar cada JSON de entrenamiento
corpus_json = df_train['json_target'].tolist()
corpus_tokenized = [word_tokenize(doc.lower()) for doc in corpus_json]

# Crear el índice BM25
bm25 = BM25Okapi(corpus_tokenized)

# Función de recuperación de ejemplo más similar
def retrieve_similar_example(ocr_input_text: str, top_k: int = 1) -> str:
    """
    Recupera el JSON estructurado más similar al texto OCR de entrada
    usando BM25 sobre el conjunto de entrenamiento.
    
    Parámetros:
    - ocr_input_text: texto plano extraído por OCR
    - top_k: número de ejemplos a recuperar (por defecto 1)
    
    Retorna:
    - Cadena JSON del ejemplo más similar
    """
    tokens = word_tokenize(ocr_input_text.lower())
    scores = bm25.get_scores(tokens)
    best_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
    retrieved_jsons = [corpus_json[i] for i in best_indices]
    return retrieved_jsons[0] if top_k == 1 else retrieved_jsons

# Prueba de funcionamiento
print("\nRecuperando JSON similar para una muestra de texto OCR...")
sample_ocr = df_val.iloc[0]['ocr_text']
retrieved_json = retrieve_similar_example(sample_ocr)

print("\nTexto OCR (val):")
print(sample_ocr[:300], '...')

print("\nJSON recuperado desde base de entrenamiento:")
print(retrieved_json)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mnico\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Recuperando JSON similar para una muestra de texto OCR...

Texto OCR (val):
Invoice no: 41888239 Date of issue: 08/11/2018 Seller: Client: Snyder-Singleton Stewart PLC 00711 Lisa Walk Suite 525 19905 Michael Drive Apt. 892 Jonesfurt, RI 73831 Rushberg, KY 36515 Tax Id: 977-95-1354 Tax Id: 925-76-5239 IBAN: GB3IIDIC80119873375691 ITEMS No. Description Qty UM Net price Net wo ...

JSON recuperado desde base de entrenamiento:

{
  "invoice": {
    "client_name": "Hansen, Campbell and Boyd",
    "client_address": "39276 Hawkins Corners\nTinabury, ID 31296",
    "seller_name": "Parrish, Anderson and Banks",
    "seller_address": "44419 Ryan Loaf\nNorth Michael, TN 05085",
    "invoice_number": "72026666",
    "invoice_date": "12/04/2012",
    "due_date": ""
  },
  "items": [
    {
      "description": "Cottage Crafts Marble Dinning\nTable Top Round Conference\nTable Size 60 Inches",
      "quantity": "3,00",
      "total_price": "18 466,80"
    },
    {
      "description": "6'x3' White Ma

## Bloque 5: Tokenización y Preprocesamiento

Descripción breve:
Este bloque se encarga de tokenizar las entradas de texto para el modelo DistilBERT, combinando el texto OCR con su contexto recuperado (usado en la arquitectura RAG). Se asegura el formato correcto mediante truncamiento, padding y batch formatting. El resultado son pares input_ids, attention_mask y labels listos para entrenamiento supervisado.

In [25]:
# Cargar el tokenizer de DistilBERT
from transformers import DistilBertTokenizerFast

# Tokenizador preentrenado
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# Parámetros de tokenización
MAX_LENGTH = 512  # Limite estándar de DistilBERT

# Función para preparar los datos
def preprocess_example(ocr_text: str, target_json: str) -> dict:
    """
    Combina el texto OCR y el contexto recuperado para crear la entrada del modelo.
    Retorna los inputs codificados listos para entrenamiento.
    """
    # Recuperar ejemplo similar desde el índice (BM25)
    retrieved_context = retrieve_similar_example(ocr_text)

    # Concatenar texto OCR + contexto como entrada
    model_input = ocr_text.strip() + "\n\n" + retrieved_context.strip()

    # Tokenizar entrada y salida
    inputs = tokenizer(
        model_input,
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    labels = tokenizer(
        target_json,
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Reestructurar para salida como diccionario plano
    return {
        "input_ids": inputs.input_ids.squeeze(),
        "attention_mask": inputs.attention_mask.squeeze(),
        "labels": labels.input_ids.squeeze()
    }

# Probar con un ejemplo
print("\nVerificando tokenización de ejemplo:")
example = preprocess_example(df_train.iloc[0]['ocr_text'], df_train.iloc[0]['json_target'])
print(f"Input IDs: {example['input_ids'][:10]}")
print(f"Labels IDs: {example['labels'][:10]}")



Verificando tokenización de ejemplo:
Input IDs: tensor([  101,  1999,  6767,  6610,  2053,  1024, 14748,  2575,  2581,  2683])
Labels IDs: tensor([ 101, 1063, 1000, 1999, 6767, 6610, 1000, 1024, 1063, 1000])


## Bloque 6: Entrenamiento del Modelo (DistilBERT + RAG)

Descripción breve:
Este bloque define un modelo encoder-decoder utilizando DistilBERT como codificador y GPT2 como decodificador. El modelo es entrenado para tomar como entrada la combinación de texto OCR + contexto recuperado, y generar como salida el JSON estructurado correspondiente. Se entrena con Trainer de Hugging Face usando los datos tokenizados previamente.

In [21]:
!pip install accelerate

In [23]:
# Instalar librería rank_bm25
!pip install rank_bm25


In [26]:
# IMPORTANTE: Asegúrate de ejecutar esto solo una vez
from rank_bm25 import BM25Okapi


# Modelo encoder-decoder: DistilBERT + GPT2
from transformers import EncoderDecoderModel

# GPT2 necesita tokens especiales definidos
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

# Descargar tokenizadores
encoder_tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
decoder_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# Cargar modelo encoder-decoder
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder_pretrained_model_name_or_path="distilbert-base-uncased",
    decoder_pretrained_model_name_or_path="gpt2"
)

# Configurar los tokens especiales requeridos por GPT2
model.config.decoder_start_token_id = decoder_tokenizer.bos_token_id
model.config.pad_token_id = encoder_tokenizer.pad_token_id

# Limitar secuencia
model.config.max_length = 512
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True

# Dataset tokenizado para entrenamiento (conversión a Dataset HuggingFace)
from datasets import Dataset

# Tokenizar los datasets completos usando la función del Bloque 5
train_encoded = [preprocess_example(row['ocr_text'], row['json_target']) for idx, row in df_train.iterrows()]
val_encoded = [preprocess_example(row['ocr_text'], row['json_target']) for idx, row in df_val.iterrows()]

train_dataset = Dataset.from_dict({
    'input_ids': [e['input_ids'] for e in train_encoded],
    'attention_mask': [e['attention_mask'] for e in train_encoded],
    'labels': [e['labels'] for e in train_encoded]
})

val_dataset = Dataset.from_dict({
    'input_ids': [e['input_ids'] for e in val_encoded],
    'attention_mask': [e['attention_mask'] for e in val_encoded],
    'labels': [e['labels'] for e in val_encoded]
})

# Argumentos de entrenamiento
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./outputs_distilbert_gpt2/",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none",  # evita errores si no usas wandb
    save_total_limit=1,
    push_to_hub=False
)

# Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=encoder_tokenizer
)

# Entrenamiento
trainer.train()


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['transformer.h.0.crossattention.c_attn.bias', 'transformer.h.0.crossattention.c_attn.weight', 'transformer.h.0.crossattention.c_proj.bias', 'transformer.h.0.crossattention.c_proj.weight', 'transformer.h.0.crossattention.q_attn.bias', 'transformer.h.0.crossattention.q_attn.weight', 'transformer.h.0.ln_cross_attn.bias', 'transformer.h.0.ln_cross_attn.weight', 'transformer.h.1.crossattention.c_attn.bias', 'transformer.h.1.crossattention.c_attn.weight', 'transformer.h.1.crossattention.c_proj.bias', 'transformer.h.1.crossattention.c_proj.weight', 'transformer.h.1.crossattention.q_attn.bias', 'transformer.h.1.crossattention.q_attn.weight', 'transformer.h.1.ln_cross_attn.bias', 'transformer.h.1.ln_cross_attn.weight', 'transformer.h.10.crossattention.c_attn.bias', 'transformer.h.10.crossattention.c_attn.weight', 'transformer.h.10.crossattention.c_proj.bias', 'transformer.h.10.cros

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [28]:
import transformers
print(transformers.__file__)


c:\Users\mnico\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\__init__.py
